In [1]:
#형태소 기준 점수내기+조건 / input 형식->mp3

import requests
import json
import re, string
import pandas as pd
import os

from konlpy.tag import Okt
okt=Okt()
punc_regex = re.compile('[{}]'.format(re.escape(string.punctuation)))

class ClovaSpeechClient:
    # Clova Speech invoke URL
    invoke_url = 'https://clovaspeech-gw.ncloud.com/external/v1/3069/d51a11e64d144c234186002d2845ade3b6e6899933fdd933834329db75f41343'
    # Clova Speech secret key
    secret = 'b13413f393ac4f41a94fae7be1ef8389'

    def req_url(self, url, completion, callback=None, userdata=None, forbiddens=None, boostings=None, wordAlignment=True, fullText=True, diarization=None):
        request_body = {
            'url': url,
            'language': 'ko-KR',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'Content-Type': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        return requests.post(headers=headers,
                             url=self.invoke_url + '/recognizer/url',
                             data=json.dumps(request_body).encode('UTF-8'))

    def req_object_storage(self, data_key, completion, callback=None, userdata=None, forbiddens=None, boostings=None,
                           wordAlignment=True, fullText=True, diarization=None):
        request_body = {
            'dataKey': data_key,
            'language': 'ko-KR',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'Content-Type': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        return requests.post(headers=headers,
                             url=self.invoke_url + '/recognizer/object-storage',
                             data=json.dumps(request_body).encode('UTF-8'))

    def req_upload(self, file, completion, callback=None, userdata=None, forbiddens=None, boostings=None,
                   wordAlignment=True, fullText=True, diarization=None):
        request_body = {
            'language': 'ko-KR',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
#         print(json.dumps(request_body, ensure_ascii=False).encode('UTF-8'))
        files = {
            'media': open(file, 'rb'),
            'params': (None, json.dumps(request_body, ensure_ascii=False).encode('UTF-8'), 'application/json')
        }
        response = requests.post(headers=headers, url=self.invoke_url + '/recognizer/upload', files=files)
        return response

def strip_punc(corpus):
    return punc_regex.sub('', corpus)  

if __name__ == '__main__':
    # res = ClovaSpeechClient().req_url(url='http://example.com/media.mp3', completion='sync')
    # res = ClovaSpeechClient().req_object_storage(data_key='data/media.mp3', completion='sync')
    res = ClovaSpeechClient().req_upload(file='./part6_30초.mp3', completion='sync')
    result = json.loads(res.text)
    result=result["text"]
    result=okt.normalize(result)
    result=strip_punc(result)
    result_text=(okt.morphs(result))
    result_noun=okt.nouns(result)
    a=len(result_text)
    
for i,document in enumerate(result):
      # okt=konlpy.tag.Okt()
    result_tag=[]
    for word in okt.pos(result):
        if word[1] in ['Josa', 'Eomi', 'Punctuation']: #조사, 어미, 구두점 제외 
            result_tag.append(word[0])
# print(result_tag)

   
dir = '/Users/wellekim/Desktop/멀티모달_Part_06'
folders = os.listdir(dir)
    
script_lst = []
for folder in folders:
    file_dir = dir + '/' + folder + '/' + folder + '_interpolation.json'
    try:
        json_object = json.load(open(file_dir))
    except:
        pass
    st_time = {}
    for idx, dialog in enumerate(json_object['dialogue_infos']):
        st_time[idx] = dialog['start_time']
    sort_time = sorted(st_time.items(), key = lambda item: item[1])
    sort_time = [i[0] for i in sort_time]

    script = ''
    for i in sort_time:
        try:
            script += json_object['dialogue_infos'][i]['utterance']
            script = re.sub(r'[^0-9a-zA-Zㄱ-ㅣ가-힣]', '', script)
            script = strip_punc(script)
            script=okt.normalize(script)
            script_text=(okt.morphs(script))
            script_noun=okt.nouns(script)
        except:
            pass
    
for i,document in enumerate(script):
      # okt=konlpy.tag.Okt()
    script_tag=[]
    for word in okt.pos(script):
        if word[1] in ['Josa', 'Eomi', 'Punctuation']: #조사, 어미, 구두점 제외 
            script_tag.append(word[0])
            

    dic = {'script': script}
    scripts=dic["script"]
    scripti=okt.phrases(scripts)
    script_lst.append(scripti)  
    script_lst_final=sum(script_lst, [])
    b=len(script_lst_final)
        
    tag_r=0
    for i in result_tag:
        if i not in script_tag:
            tag_r+=1
            if tag_r==0:
                print("일치하므로 accuracy는 100%입니다")
    
    tag_s=0
    for i in script_tag:
        if i not in result_tag:
            tag_s+=1
            if tag_s==0:
                print("일치하므로 accuracy는 100%입니다")
     
    result_n=0
    for i in result_noun:
        if i not in script_noun:
            result_n+=1
            if result_n==0:
                print("일치하므로 accuracy는 100%입니다")
    
    scrip_n=0
    for i in script_noun:
        if i not in result_noun:
            scrip_n+=1
            if scrip_n==0:
                print("일치하므로 accuracy는 100%입니다")
    
accuracy=(a+b-((tag_r*tag_s)*20+(result_n+scrip_n)*80))/100
print(accuracy)

if accuracy<100:
    print(f"accuracy:{accuracy}%")
if accuracy>100:
    print("일치하므로 accuracy는 100%입니다")

-23.57
인식률기준의 accuracy:-23.57%
